<a href="https://colab.research.google.com/github/ambirpatel/Optimized-LLM/blob/main/vicuna_cpp_compilation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from huggingface_hub import snapshot_download

model_name = "lmsys/vicuna-7b-v1.3"
base_model = "./base_model/"
snapshot_download(repo_id=model_name, local_dir=base_model, ignore_patterns=["*.pth"])

Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/566 [00:00<?, ?B/s]

...

README.md:   0%|          | 0.00/1.98k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

'/content/base_model'

# Clone llama cpp repo
1. Convert model to GGUF format
2. Build llama cpp
3. Quantize the model

In [3]:
!git clone https://github.com/ggerganov/llama.cpp

In [4]:
!mkdir compiled_model

In [11]:
!python llama.cpp/convert_hf_to_gguf.py ./base_model/ --outfile compiled_model/vicuna_7b_FP16.gguf

In [12]:
# Build llama cpp
!mkdir llama.cpp/build && cd llama.cpp/build && cmake .. && cmake --build . --config Release

In [13]:
# quantize the model to 4-bits (using Q4_K_M method)
!cd llama.cpp/build/bin && ./llama-quantize /content/compiled_model/vicuna_7b_FP16.gguf /content/compiled_model/vicuna_7b_FP16_K_M.gguf q4_K_M

# Inference on quantized model

In [14]:
!pip install llama-cpp-python==0.2.85

In [16]:
from llama_cpp import Llama

model_path = "/content/compiled_model/vicuna_7b_FP16_K_M.gguf"

llm = Llama(model_path=model_path)

llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from /content/compiled_model/vicuna_7b_FP16_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Base_Model
llama_model_loader: - kv   3:                         general.size_label str              = 6.7B
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                       llama.context_length u32              = 2048
llama_model_loader: - kv   6:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   7:                  llama.feed_fo

In [17]:
generation_kwargs = {
    "max_tokens":200,
    "echo":False,
    "top_k":1
}

In [18]:
prompt = "Which country hosted 2018 fifa world cup?"
res = llm(prompt, **generation_kwargs)
res


llama_print_timings:        load time =    5586.80 ms
llama_print_timings:      sample time =       7.34 ms /   200 runs   (    0.04 ms per token, 27244.24 tokens per second)
llama_print_timings: prompt eval time =    5586.70 ms /    14 tokens (  399.05 ms per token,     2.51 tokens per second)
llama_print_timings:        eval time =  142702.47 ms /   199 runs   (  717.10 ms per token,     1.39 tokens per second)
llama_print_timings:       total time =  148533.93 ms /   213 tokens


{'id': 'cmpl-4d7e379d-dc7a-4427-858c-14b66b210a21',
 'object': 'text_completion',
 'created': 1729348078,
 'model': '/content/compiled_model/vicuna_7b_FP16_K_M.gguf',
 'choices': [{'text': "\nThe 2018 FIFA World Cup was held in Russia from June 14 to July 15, 2 The 2018 FIFA World Cup was the 21st FIFA World Cup, an international football tournament contested by the men's national teams of the member associations of FIFA once every four years. It took place in Russia from 14 June to 15 July 2018. It was the first World Cup to be held in Eastern Europe, and the 11th time that it had been held in Europe.\nThe tournament consisted of 32 teams, which were selected from qualifying matches held between March 2015 and October 2017. A total of 64 matches were played in 12 venues located in 11 cities across Russia. It was the first World Cup to use the video assistant referee (VAR) system.\nFrance won the tournament",
   'index': 0,
   'logprobs': None,
   'finish_reason': 'length'}],
 'usage':